<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 2: Preparing Data - Data Normalization

## Data Normalization 

### Lesson Objectives

-	After completing this lesson, you should be able to: 
-	Normalize a dataset to have unit p-norm
-	Normalize a dataset to have unit standard deviation and zero mean 
-	Normalize a dataset to have given minimum and maximum values 


## Normalizer

-	A Transformer which transforms a dataset of Vector rows, normalizing each Vector to have unit norm
-	Takes a parameter P, which specifies the p-norm used for normalization (p=2 by default)
- Standardize input data and improve the behavior of learning algorithms

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0` // Or use any other 2.x version here
import  org.apache.spark.SparkContext
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
val sc= new SparkContext("local[*]","stats")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                     // Or use any other 2.x version here

import  org.apache.spark.SparkContext

import org.apache.log4j.{Level, Logger}

sc: SparkContext = org.apache.spark.SparkContext@3fa2dbe2

In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

import org.apache.spark.sql.SparkSession

spark: SparkSession = org.apache.spark.sql.SparkSession@1ef6de3c
import spark.implicits._

In [3]:
// Continuing from Previous Example 

import  org.apache.spark.ml.feature.VectorAssembler

import  org.apache.spark.sql.functions._

val dfRandom = spark.range(0, 10).select("id").
 withColumn("uniform", rand(10L)).
 withColumn("normal1", randn(10L)).
 withColumn("normal2", randn(11L))

val assembler = new  VectorAssembler().
 setInputCols(Array("uniform","normal1","normal2")).
 setOutputCol("features")

val dfVec = assembler.transform(dfRandom)


// Continuing from Previous Example 

dfVec.select("id","features").show()
 

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.41371264720975...|
|  1|[0.73117192818966...|
|  2|[0.19829196382083...|
|  3|[0.12714181165849...|
|  4|[0.76043181534066...|
|  5|[0.12030715258495...|
|  6|[0.12131363910425...|
|  7|[0.44292918521277...|
|  8|[0.88987842538862...|
|  9|[0.03650707717266...|
+---+--------------------+



import  org.apache.spark.ml.feature.VectorAssembler


import  org.apache.spark.sql.functions._


dfRandom: org.apache.spark.sql.package.DataFrame = [id: bigint, uniform: double ... 2 more fields]
assembler: VectorAssembler = vecAssembler_54eefc605323
dfVec: org.apache.spark.sql.package.DataFrame = [id: bigint, uniform: double ... 3 more fields]

In [4]:

// A Simple Normalizer 

import  org.apache.spark.ml.feature.Normalizer

val scaler1 = new Normalizer().setInputCol("features").setOutputCol("scaledFeat").setP(1.0)
scaler1.transform(dfVec.select("id","features")).show(5)

+---+--------------------+--------------------+
| id|            features|          scaledFeat|
+---+--------------------+--------------------+
|  0|[0.41371264720975...|[0.32886636983701...|
|  1|[0.73117192818966...|[0.27877135762286...|
|  2|[0.19829196382083...|[0.20619308493718...|
|  3|[0.12714181165849...|[0.06801701322638...|
|  4|[0.76043181534066...|[0.54081735791552...|
+---+--------------------+--------------------+
only showing top 5 rows



import  org.apache.spark.ml.feature.Normalizer


scaler1: Normalizer = normalizer_aa922c1c6e2e

## Standard Scaler

-	A Model which can be fit on a dataset to produce a `StandardScalerModel`
-	A Transformer which transforms a dataset of `Vector` rows, normalizing each feature to have unit standard deviation and/or zero mean
- Takes two parameters:
	-	`withStd`: scales the data to unit standard deviation (default: true)
	-	`withMean`: centers the data with mean before scaling (default: false)
-	It builds a dense output, sparse inputs will raise an exception
-	If the standard deviation of a feature is zero, it returns 0.0 in the Vector for that feature

In [5]:
// A Simple Standard Scaler 

import  org.apache.spark.ml.feature.StandardScaler

val  scaler2 = new StandardScaler().
 setInputCol("features"). setOutputCol("scaledFeat").
 setWithStd(true). setWithMean(true)

val  scaler2Model = scaler2.fit(dfVec.select("id","features"))
scaler2Model.transform(dfVec.select("id","features")).show(5)

+---+--------------------+--------------------+
| id|            features|          scaledFeat|
+---+--------------------+--------------------+
|  0|[0.41371264720975...|[0.09436171519474...|
|  1|[0.73117192818966...|[1.10830425730255...|
|  2|[0.19829196382083...|[-0.5936767465057...|
|  3|[0.12714181165849...|[-0.8209253118403...|
|  4|[0.76043181534066...|[1.20175827215480...|
+---+--------------------+--------------------+
only showing top 5 rows



import  org.apache.spark.ml.feature.StandardScaler


scaler2: StandardScaler = stdScal_446516dbfc0c
scaler2Model: org.apache.spark.ml.feature.StandardScalerModel = stdScal_446516dbfc0c

## MinMax Scaler 

-	A Model which can be fit on a dataset to produce a `MinMaxScalerModel`
-	A Transformer which transforms a dataset of `Vector` rows, rescaling each feature to a specific range (often `[0,1]`)
-	Takes two parameters: 
	-	min: lower bound after transformation, shared by all features (default:0.0)
	-	max: upper bound after transformation, shared by all features (default: 1.0)
-	Since zero values are likely to be transformed to non-zero values, sparse inputs may result in dense outputs

In [6]:

// A Simple MinMax Scaler 
import  org.apache.spark.ml.feature.MinMaxScaler 

val scaler3 = new MinMaxScaler().
 setInputCol("features").setOutputCol("scaledFeat").
 setMin(-1.0).setMax(1.0)

val scaler3Model = scaler3.fit(dfVec.select("id","features"))
scaler3Model.transform(dfVec.select("id","features")).show(5)

+---+--------------------+--------------------+
| id|            features|          scaledFeat|
+---+--------------------+--------------------+
|  0|[0.41371264720975...|[-0.1159638278794...|
|  1|[0.73117192818966...|[0.62804821715480...|
|  2|[0.19829196382083...|[-0.6208335632866...|
|  3|[0.12714181165849...|[-0.7875843038899...|
|  4|[0.76043181534066...|[0.69662302274718...|
+---+--------------------+--------------------+
only showing top 5 rows



import  org.apache.spark.ml.feature.MinMaxScaler 


scaler3: MinMaxScaler = minMaxScal_c7767985479b
scaler3Model: org.apache.spark.ml.feature.MinMaxScalerModel = minMaxScal_c7767985479b

In [7]:
sc.stop()

## Lesson Summary 

-	Having completed this lesson, you should be able to: 
- Normalize a dataset to have unit p-norm
-	Normalize a dataset to have unit standard deviation and zero mean
-	Normalize a dataset to have given minimum and maximum values

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.